In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

data = pd.read_csv("E:/R(5)/Dataset_5.csv")
X = data.drop('label', axis=1)
y = data['label']

split_ratios = [0.7, 0.8, 0.9]

accuracy_results = []
confusion_matrix_results = []
auc_results = []
training_testing_times = []

for ratio in split_ratios:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - ratio, random_state=42)

    classifiers = {
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'Naive Bayes': GaussianNB(),
        'Support Vector Machine': SVC(probability=True),
        'XGBoost': XGBClassifier(random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42)
    }

    for clf_name, clf in classifiers.items():
        start_time = time.time()
        clf.fit(X_train, y_train)
        end_time = time.time()
        training_time = end_time - start_time

        start_time = time.time()
        y_pred = clf.predict(X_test)
        end_time = time.time()
        testing_time = end_time - start_time
        
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_results.append({
            'Split Ratio': ratio,
            'Classifier': clf_name,
            'Accuracy': accuracy
        })
        
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        tpr = tp / (tp + fn)
        fnr = fn / (tp + fn)
        tnr = tn / (tn + fp)
        fpr = fp / (tn + fp)
        confusion_matrix_results.append({
            'Split Ratio': ratio,
            'Classifier': clf_name,
            'True Positive': tp,
            'False Negative': fn,
            'True Negative': tn,
            'False Positive': fp,
            'True Positive Rate': tpr,
            'False Negative Rate': fnr,
            'True Negative Rate': tnr,
            'False Positive Rate': fpr
        })
        
        try:
            probas = clf.predict_proba(X_test)
            auc_value = roc_auc_score(y_test, probas[:, 1])
        except AttributeError:
            auc_value = None
        auc_results.append({
            'Split Ratio': ratio,
            'Classifier': clf_name,
            'AUC Value': auc_value
        })

        training_testing_times.append({
            'Split Ratio': ratio,
            'Classifier': clf_name,
            'Training Time (s)': training_time,
            'Testing Time (s)': testing_time
        })

accuracy_df = pd.DataFrame(accuracy_results)
confusion_matrix_df = pd.DataFrame(confusion_matrix_results)
auc_df = pd.DataFrame(auc_results)
training_testing_times_df = pd.DataFrame(training_testing_times)

In [3]:
print("Accuracy:")
accuracy_df

Accuracy:


,Split Ratio,Classifier,Accuracy
0,0.7,Random Forest,1.000000
1,0.7,Gradient Boosting,0.991736
2,0.7,AdaBoost,1.000000
3,0.7,Naive Bayes,0.942149
4,0.7,Support Vector Machine,0.991736
5,0.7,XGBoost,0.983471
6,0.7,Decision Tree,0.991736
7,0.8,Random Forest,1.000000
8,0.8,Gradient Boosting,0.987500
9,0.8,AdaBoost,0.987500


In [4]:
print("\nConfusion Matrix Metrics:")
confusion_matrix_df


Confusion Matrix Metrics:


,Split Ratio,Classifier,True Positive,False Negative,True Negative,False Positive,True Positive Rate,False Negative Rate,True Negative Rate,False Positive Rate
0,0.7,Random Forest,77,0,44,0,1.000000,0.000000,1.000000,0.000000
1,0.7,Gradient Boosting,76,1,44,0,0.987013,0.012987,1.000000,0.000000
2,0.7,AdaBoost,77,0,44,0,1.000000,0.000000,1.000000,0.000000
3,0.7,Naive Bayes,72,5,42,2,0.935065,0.064935,0.954545,0.045455
4,0.7,Support Vector Machine,76,1,44,0,0.987013,0.012987,1.000000,0.000000
5,0.7,XGBoost,76,1,43,1,0.987013,0.012987,0.977273,0.022727
6,0.7,Decision Tree,76,1,44,0,0.987013,0.012987,1.000000,0.000000
7,0.8,Random Forest,52,0,28,0,1.000000,0.000000,1.000000,0.000000
8,0.8,Gradient Boosting,51,1,28,0,0.980769,0.019231,1.000000,0.000000
9,0.8,AdaBoost,51,1,28,0,0.980769,0.019231,1.000000,0.000000


In [5]:
print("\nAUC Values:")
auc_df


AUC Values:


,Split Ratio,Classifier,AUC Value
0,0.7,Random Forest,1.000000
1,0.7,Gradient Boosting,1.000000
2,0.7,AdaBoost,1.000000
3,0.7,Naive Bayes,0.972550
4,0.7,Support Vector Machine,1.000000
5,0.7,XGBoost,0.999410
6,0.7,Decision Tree,0.993506
7,0.8,Random Forest,1.000000
8,0.8,Gradient Boosting,1.000000
9,0.8,AdaBoost,1.000000


In [6]:
print("\nTraining Times:")
training_testing_times_df


Training Times:


,Split Ratio,Classifier,Training Time (s),Testing Time (s)
0,0.7,Random Forest,0.328123,0.025930
1,0.7,Gradient Boosting,0.289738,0.002991
2,0.7,AdaBoost,0.185506,0.020944
3,0.7,Naive Bayes,0.002991,0.001994
4,0.7,Support Vector Machine,0.015955,0.003990
5,0.7,XGBoost,0.070810,0.004986
6,0.7,Decision Tree,0.005984,0.002992
7,0.8,Random Forest,0.325668,0.028368
8,0.8,Gradient Boosting,0.277259,0.003990
9,0.8,AdaBoost,0.172431,0.024935
